Author: Roel Faber

In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

Start automatic browser and go to website with football statistics

In [2]:
browser = webdriver.Chrome()
browser.get('https://www.voetbal.com/wedstrijd/ned-eredivisie-2019-2020/')

Don't agree to use cookies

In [4]:
browser.find_element_by_class_name("qc-cmp-button").click()
browser.find_element_by_class_name("qc-cmp-save-and-exit").click()

In [6]:
data = requests.get(browser.current_url)

In [7]:
soup = BeautifulSoup(data.text,'html.parser')

In [8]:
soup

<!DOCTYPE html>

<html class="ad-version-nl" lang="nl" xml:lang="nl" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://ogp.me/ns/fb#">
<head>
<title>Eredivisie 2019/2020</title>
<meta content="Eredivisie 2019/2020: Actuele berichten, Programma en Resultaten, Stand, Teams, Topscorers. De in een oogopslag Competitie..." name="description"/>
<meta content="Eredivisie, Voetbal, 2019/2020, Nieuws, Actuele berichten, Programma en Resultaten, Stand, Teams, Topscorers, forum" name="keywords"/>
<meta content="Eredivisie" name="page-topic"/>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<meta content="nl" http-equiv="content-language">
<meta content="nl_NL" property="og:locale">
<meta content="Eredivisie 2019/2020" property="og:title">
<meta content="website" property="og:type"/>
<meta content="https://s.hs-data.com/bilder/wettbewerbe/100/203.gif" property="og:image"/>
<meta content="//www.voetbal.com/wedstrijd/ned-eredivisie-2019-2020/" property